In [15]:
'''
This notebook is just mocking the main.py in the fixed version simpletod repository (https://github.com/searchforpassion/simpletod) for playing around with SimpleTOD. 
To run the notebook, put this file beside the main.py of the simpletod repo root folder.
Also make sure you have run the create_dataset.sh
'''

'\nThis notebook is just mocking the main.py in the fixed version simpletod repository (https://github.com/searchforpassion/simpletod) for playing around with SimpleTOD. \nTo run the notebook, put this file beside the main.py of the simpletod repo root folder.\nAlso make sure you have run the create_dataset.sh\n'

In [16]:
import sys

sys.argv = [
    "main.py",
    "--output_dir=output/gpt2",
    "--model_type=gpt2-small",
    "--model_name_or_path=gpt2",
    "--do_train",
    "--train_data_file=./resources/gpt2/train.history_belief",
    "--do_eval",
    "--eval_data_file=./resources/gpt2/val.history_belief",
    "--evaluate_during_training",
    "--save_steps",
    "1000000000",
    "--logging_steps",
    "1000000000",
    "--per_gpu_train_batch_size",
    "1",
    "--num_train_epochs",
    "1",
]


import argparse
import glob
import logging
import os
import pickle
import random
import re

# import shutil
# from typing import Dict, List, Tuple

import numpy as np
import torch

# from torch.nn.utils.rnn import pad_sequence
# from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
# from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from transformers import (
    WEIGHTS_NAME,
    # AdamW,
    GPT2Tokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    # get_linear_schedule_with_warmup,
)

from transformers import GPT2Tokenizer

# comment this if you want to load gpt2 class from transformers
from models import GPT2LMHeadModel
from models import GPT2Config, GPT2SmallConfig

# uncomment this if you want to load gpt2 class from transformers
# from transformers import GP2Config, GPT2LMHeadModel

from data.dataset.language_model import *
from utils.model import *
from utils.language_model import get_optimizer_scheduler
from utils.gpt2_args_parser import ArgsParser

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

logger = logging.getLogger(__name__)


MODEL_CLASSES = {
    "gpt2": (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),
    "gpt2-small": (GPT2SmallConfig, GPT2LMHeadModel, GPT2Tokenizer),
}

In [17]:
torch.cuda.empty_cache()

In [18]:
torch.cuda.device_count()

6

In [19]:
args = ArgsParser().parse()
print(args)

Namespace(adam_epsilon=1e-08, block_size=-1, cache_dir=None, config_name=None, do_eval=True, do_train=True, eval_all_checkpoints=False, eval_data_file='./resources/gpt2/val.history_belief', evaluate_during_training=True, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=1, learning_rate=5e-05, local_rank=-1, logging_steps=1000000000, max_grad_norm=1.0, max_steps=-1, model_name_or_path='gpt2', model_type='gpt2-small', no_cuda=False, num_train_epochs=1.0, output_dir='output/gpt2', overwrite_cache=False, per_gpu_eval_batch_size=4, per_gpu_train_batch_size=1, save_steps=1000000000, save_total_limit=None, seed=42, should_continue=False, shuffle_belief=False, shuffle_belief_action=False, shuffle_context=False, tokenizer_name=None, train_data_file='./resources/gpt2/train.history_belief', warmup_steps=0, weight_decay=0.0)


In [20]:
if args.eval_data_file is None and args.do_eval:
    raise ValueError(
        "--eval_data_file should be specified when do_eval is true"
    )
if args.should_continue:
    sorted_checkpoints = _sorted_checkpoints(args)
    if len(sorted_checkpoints) == 0:
        raise ValueError(
            "--should_continue is true, but no checkpoint found in --output_dir"
        )
    else:
        args.model_name_or_path = sorted_checkpoints[-1]

In [21]:
# Setup CUDA, GPU & distributed training
if args.local_rank == -1 or args.no_cuda:
    device = torch.device(
        "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
    )
    args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()
else:  # initialize distributed training
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    torch.distributed.init_process_group(backend="nccl")
    args.n_gpu = 1
args.device = device
device

device(type='cuda')

In [22]:
# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
)
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    args.local_rank,
    device,
    args.n_gpu,
    bool(args.local_rank != -1),
    args.fp16,
)

In [23]:
def get_model_tokenizer(args):
    config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]

    if args.config_name:
        config = config_class.from_pretrained(
            args.config_name, cache_dir=args.cache_dir
        )
    elif args.model_name_or_path:
        config = config_class.from_pretrained(
            args.model_name_or_path, cache_dir=args.cache_dir
        )
    else:
        config = config_class()

    if args.tokenizer_name:
        tokenizer = tokenizer_class.from_pretrained(
            args.tokenizer_name, cache_dir=args.cache_dir
        )
    elif args.model_name_or_path:
        tokenizer = tokenizer_class.from_pretrained(
            args.model_name_or_path, cache_dir=args.cache_dir
        )
    else:
        raise ValueError(
            "You are instantiating a new {} tokenizer. This is not supported, but you can do it from another script, save it,"
            "and load it from here, using --tokenizer_name".format(
                tokenizer_class.__name__
            )
        )

    if args.block_size <= 0:
        args.block_size = tokenizer.model_max_length
        # Our input block size will be the max possible for the model
    else:
        args.block_size = min(args.block_size, tokenizer.model_max_length)

    if args.model_name_or_path:
        model = model_class.from_pretrained(
            args.model_name_or_path,
            from_tf=bool(".ckpt" in args.model_name_or_path),
            config=config,
            cache_dir=args.cache_dir,
        )
    else:
        logger.info("Training new model from scratch")
        model = model_class(config=config)

    model.to(args.device)

    if args.model_name_or_path == "openai-gpt":
        tokenizer.add_special_tokens({"bos_token": "<|endoftext|>"})
        tokenizer.add_special_tokens({"eos_token": "<|endoftext|>"})
    elif args.model_name_or_path == "gpt2":
        pass

    return model, tokenizer, model_class, args

In [24]:
# Load pretrained model and tokenizer
if args.local_rank not in [-1, 0]:
    torch.distributed.barrier()  # if not the first process, do not load pretrained model & vocab

model, tokenizer, model_class, args = get_model_tokenizer(args)

if args.local_rank == 0:
    torch.distributed.barrier()  # finish barrier, when first process has loaded pretrained model & vocab

In [ ]:
train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)

In [ ]:
print("Train Size", len(train_dataset))
print("Example", train_dataset[0])

In [ ]:
def train(args, train_dataset, model, tokenizer):
    """Train the model"""
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter(
            "./runs/{}".format(args.output_dir.split("/")[-1])
        )

    # Prepare dataloader
    train_dataloader, args = get_dataloader(train_dataset, tokenizer, args)

    # total iteration and batch size
    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = (
            args.max_steps
            // (len(train_dataloader) // args.gradient_accumulation_steps)
            + 1
        )
    else:
        t_total = (
            len(train_dataloader)
            // args.gradient_accumulation_steps
            * args.num_train_epochs
        )

    total_batch_size = (
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1)
    )

    # Prepare optimizer and schedule (linear warmup and decay)
    optimizer, scheduler = get_optimizer_scheduler(args, model, t_total)

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use fp16 training."
            )
        model, optimizer = amp.initialize(
            model, optimizer, opt_level=args.fp16_opt_level
        )

    # multi-gpu training
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model,
            device_ids=[args.local_rank],
            output_device=args.local_rank,
            find_unused_parameters=True,
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = {}".format(len(train_dataset)))
    logger.info("  Num Epochs = {}".format(args.num_train_epochs))
    logger.info(
        "  Instantaneous batch size per GPU = {}".format(
            args.per_gpu_train_batch_size
        )
    )
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = {}".format(
            total_batch_size
        )
    )
    logger.info(
        "  Gradient Accumulation steps = {}".format(
            args.gradient_accumulation_steps
        )
    )
    logger.info("  Total optimization steps = {}".format(t_total))

    (
        global_step,
        epochs_trained,
        steps_trained_in_current_epoch,
    ) = get_training_info(train_dataloader, args)

    tr_loss, logging_loss = 0.0, 0.0

    model_to_resize = (
        model.module if hasattr(model, "module") else model
    )  # Take care of distributed/parallel training
    model_to_resize.resize_token_embeddings(len(tokenizer))

    model.zero_grad()

    train_iterator = trange(
        epochs_trained,
        int(args.num_train_epochs),
        desc="Epoch",
        disable=args.local_rank not in [-1, 0],
    )

    for _ in train_iterator:

        (
            model,
            optimizer,
            scheduler,
            global_step,
            tr_loss,
            logging_loss,
        ) = train_epoch(
            model,
            tokenizer,
            optimizer,
            scheduler,
            train_dataloader,
            tr_loss,
            logging_loss,
            global_step,
            steps_trained_in_current_epoch,
            tb_writer,
            args,
        )

        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

In [ ]:
def get_training_info(dataloader, args):
    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0

    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split(
                "/"
            )[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (
                len(dataloader) // args.gradient_accumulation_steps
            )
            steps_trained_in_current_epoch = global_step % (
                len(dataloader) // args.gradient_accumulation_steps
            )

            logger.info(
                "  Continuing training from checkpoint, will skip to saved global_step"
            )
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info(
                "  Continuing training from global step %d", global_step
            )
            logger.info(
                "  Will skip the first %d steps in the first epoch",
                steps_trained_in_current_epoch,
            )
        except ValueError:
            logger.info("  Starting fine-tuning.")
    return global_step, epochs_trained, steps_trained_in_current_epoch


def train_epoch(
    model,
    tokenizer,
    optimizer,
    scheduler,
    train_dataloader,
    tr_loss,
    logging_loss,
    global_step,
    steps_trained_in_current_epoch,
    tb_writer,
    args,
):
    """train one epoch"""
    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use fp16 training."
            )

    epoch_iterator = tqdm(
        train_dataloader,
        desc="Iteration",
        disable=args.local_rank not in [-1, 0],
    )
    for step, batch in enumerate(epoch_iterator):

        # Skip past any already trained steps if resuming training
        if steps_trained_in_current_epoch > 0:
            steps_trained_in_current_epoch -= 1
            continue

        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)
        model.train()
        outputs = model(inputs, labels=labels)
        loss = outputs[
            0
        ]  # model outputs are always tuple in transformers (see doc)

        if args.n_gpu > 1:
            loss = (
                loss.mean()
            )  # mean() to average on multi-gpu parallel training
        if args.gradient_accumulation_steps > 1:
            loss = loss / args.gradient_accumulation_steps

        if args.fp16:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            loss.backward()

        tr_loss += loss.item()
        if (step + 1) % args.gradient_accumulation_steps == 0:
            if args.fp16:
                torch.nn.utils.clip_grad_norm_(
                    amp.master_params(optimizer), args.max_grad_norm
                )
            else:
                torch.nn.utils.clip_grad_norm_(
                    model.parameters(), args.max_grad_norm
                )
            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            model.zero_grad()
            global_step += 1

            # Log metrics
            if (
                args.local_rank in [-1, 0]
                and args.logging_steps > 0
                and global_step % args.logging_steps == 0
            ):
                if (
                    args.local_rank == -1 and args.evaluate_during_training
                ):  # Only evaluate when single GPU otherwise metrics may not average well
                    results = evaluate(args, model, tokenizer)
                    for key, value in results.items():
                        tb_writer.add_scalar(
                            "eval_{}".format(key), value, global_step
                        )
                tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                tb_writer.add_scalar(
                    "loss",
                    (tr_loss - logging_loss) / args.logging_steps,
                    global_step,
                )
                logging_loss = tr_loss

            # save checkpoint
            if (
                args.local_rank in [-1, 0]
                and args.save_steps > 0
                and global_step % args.save_steps == 0
            ):
                if args.evaluate_during_training:
                    save_checkpoint(
                        model,
                        optimizer,
                        scheduler,
                        tokenizer,
                        args,
                        global_step,
                    )

        if args.max_steps > 0 and global_step > args.max_steps:
            epoch_iterator.close()
            break

    return model, optimizer, scheduler, global_step, tr_loss, logging_loss


def train(args, train_dataset, model, tokenizer):
    """Train the model"""
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter(
            "./runs/{}".format(args.output_dir.split("/")[-1])
        )

    # Prepare dataloader
    train_dataloader, args = get_dataloader(train_dataset, tokenizer, args)

    # total iteration and batch size
    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = (
            args.max_steps
            // (len(train_dataloader) // args.gradient_accumulation_steps)
            + 1
        )
    else:
        t_total = (
            len(train_dataloader)
            // args.gradient_accumulation_steps
            * args.num_train_epochs
        )

    total_batch_size = (
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1)
    )

    # Prepare optimizer and schedule (linear warmup and decay)
    optimizer, scheduler = get_optimizer_scheduler(args, model, t_total)

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError(
                "Please install apex from https://www.github.com/nvidia/apex to use fp16 training."
            )
        model, optimizer = amp.initialize(
            model, optimizer, opt_level=args.fp16_opt_level
        )

    # multi-gpu training
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model,
            device_ids=[args.local_rank],
            output_device=args.local_rank,
            find_unused_parameters=True,
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = {}".format(len(train_dataset)))
    logger.info("  Num Epochs = {}".format(args.num_train_epochs))
    logger.info(
        "  Instantaneous batch size per GPU = {}".format(
            args.per_gpu_train_batch_size
        )
    )
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = {}".format(
            total_batch_size
        )
    )
    logger.info(
        "  Gradient Accumulation steps = {}".format(
            args.gradient_accumulation_steps
        )
    )
    logger.info("  Total optimization steps = {}".format(t_total))

    (
        global_step,
        epochs_trained,
        steps_trained_in_current_epoch,
    ) = get_training_info(train_dataloader, args)

    tr_loss, logging_loss = 0.0, 0.0

    model_to_resize = (
        model.module if hasattr(model, "module") else model
    )  # Take care of distributed/parallel training
    model_to_resize.resize_token_embeddings(len(tokenizer))

    model.zero_grad()

    train_iterator = trange(
        epochs_trained,
        int(args.num_train_epochs),
        desc="Epoch",
        disable=args.local_rank not in [-1, 0],
    )

    for _ in train_iterator:

        (
            model,
            optimizer,
            scheduler,
            global_step,
            tr_loss,
            logging_loss,
        ) = train_epoch(
            model,
            tokenizer,
            optimizer,
            scheduler,
            train_dataloader,
            tr_loss,
            logging_loss,
            global_step,
            steps_trained_in_current_epoch,
            tb_writer,
            args,
        )

        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step


def evaluate(args, model, tokenizer, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, evaluate=True)

    if args.local_rank in [-1, 0]:
        os.makedirs(eval_output_dir, exist_ok=True)

    # Prepare dataloader
    eval_dataloader, args = get_dataloader(
        eval_dataset, tokenizer, args, split="eval"
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = {}".format(len(eval_dataset)))
    logger.info("  Batch size = {}".format(args.eval_batch_size))
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(
        eval_output_dir, prefix, "eval_results.txt"
    )
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [14]:
global_step, train_loss = train(args, train_dataset, model, tokenizer)

Iteration:   0%|          | 0/9463 [00:00<?, ?it/s]/data/users/cting3/miniconda3/envs/py_3_8/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '

Epoch:   0%|          | 0/1 [00:51<?, ?it/s]


KeyboardInterrupt: 

In [15]:
results = {}
if args.do_eval and args.local_rank in [-1, 0]:
    checkpoints = [args.output_dir]

if args.eval_all_checkpoints:
    checkpoints = list(
        os.path.dirname(c)
        for c in sorted(
            glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True)
        )
    )
    logging.getLogger("models.modeling_utils").setLevel(
        logging.WARN
    )  # Reduce logging
logger.info("Evaluate the following checkpoints: {}".format(checkpoints))

for checkpoint in checkpoints:
    global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
    prefix = (
        checkpoint.split("/")[-1]
        if checkpoint.find("checkpoint") != -1
        else ""
    )

    model = model_class.from_pretrained(checkpoint)
    model.to(args.device)
    result = evaluate(args, model, tokenizer, prefix=prefix)
    result = dict(
        (k + "_{}".format(global_step), v) for k, v in result.items()
    )
    results.update(result)

Evaluating:  42%|████▏     | 130/308 [02:19<03:11,  1.08s/it]


KeyboardInterrupt: 

In [15]:
results, checkpoints

NameError: name 'results' is not defined

In [16]:
eval_output_dir = args.output_dir

eval_dataset = load_and_cache_examples(args, tokenizer, evaluate=True)

In [17]:
eval_dataloader, args = get_dataloader(
    eval_dataset, tokenizer, args, split="eval"
)

In [18]:


for batch in eval_dataloader:
    print(batch.shape)
    inputs, labels = (batch, batch)
    inputs = inputs.to(args.device)
    labels = labels.to(args.device)
    outputs = model(inputs, labels=labels)
    preds = torch.argmax(outputs[1], dim=2)

    input_tokens = [
        [tokenizer.decode(token.item()) for token in input] for input in inputs
    ]
    output_tokens = [
        [tokenizer.decode(token.item()) for token in pred] for pred in preds
    ]

    for i in range(len(input_tokens)):
        print(input_tokens[i])
        print(output_tokens[i])
        print("==========================")

    break

torch.Size([24, 515])


RuntimeError: CUDA out of memory. Tried to allocate 146.00 MiB (GPU 0; 23.70 GiB total capacity; 21.74 GiB already allocated; 106.56 MiB free; 22.07 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF